In [ ]:
!pip install tensorflow-text

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d masoud3/epinions-trust-network

In [1]:
from scipy.sparse import csr_matrix
import tensorflow as tf 
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from tensorflow.keras import datasets, layers, models

In [ ]:
!unzip "/content/epinions-trust-network.zip" -d "/content/"

In [2]:
df_raw = pd.read_csv('/content/epinions/rating.txt',delimiter="\t")
print(df_raw.shape)
df_raw.head()

(13668319, 8)


,139431556,591156,5,0,2001/01/10,Unnamed: 5,1,2518365
0,139431556,1312460676,5,0,2001/01/10,NaN,1,2518365.0
1,139431556,204358,5,0,2001/01/10,NaN,1,2518365.0
2,139431556,368725,5,0,2001/01/10,NaN,1,2518365.0
3,139431556,277629,5,0,2001/01/10,NaN,1,2518365.0
4,139431556,246386,5,0,2001/01/10,NaN,1,2518365.0


In [3]:
df = df_raw.dropna(subset=['5'],axis=0)
print(df.shape)
df.head()

(13668319, 8)


,139431556,591156,5,0,2001/01/10,Unnamed: 5,1,2518365
0,139431556,1312460676,5,0,2001/01/10,NaN,1,2518365.0
1,139431556,204358,5,0,2001/01/10,NaN,1,2518365.0
2,139431556,368725,5,0,2001/01/10,NaN,1,2518365.0
3,139431556,277629,5,0,2001/01/10,NaN,1,2518365.0
4,139431556,246386,5,0,2001/01/10,NaN,1,2518365.0


In [4]:
df.to_csv('/content/epinions/rating.csv')
df.head()

,139431556,591156,5,0,2001/01/10,Unnamed: 5,1,2518365
0,139431556,1312460676,5,0,2001/01/10,NaN,1,2518365.0
1,139431556,204358,5,0,2001/01/10,NaN,1,2518365.0
2,139431556,368725,5,0,2001/01/10,NaN,1,2518365.0
3,139431556,277629,5,0,2001/01/10,NaN,1,2518365.0
4,139431556,246386,5,0,2001/01/10,NaN,1,2518365.0


In [5]:
df.columns = ["Item_ID","User_ID","rating","a","b","c","d","e"]
df = df.drop(["a","b","c","d","e"],axis=1)

In [6]:
df.head()

,Item_ID,User_ID,rating
0,139431556,1312460676,5
1,139431556,204358,5
2,139431556,368725,5
3,139431556,277629,5
4,139431556,246386,5


In [7]:
df.shape

(13668319, 3)

In [8]:
df = df.drop_duplicates(subset =['Item_ID', 'User_ID'] , keep = 'first')

In [9]:
df_test = df[:500000]

In [10]:
df_test.shape

(500000, 3)

In [11]:
df_test.to_csv('/content/epinions/epinions_rating_500k.csv')

In [17]:
df_test.groupby("Item_ID").count()

,User_ID,rating
Item_ID,,
719444,3,3
719451,208,208
719455,5,5
719473,30,30
719488,22,22
...,...,...
4362505860,4,4
4363292292,11,11
4363882116,4,4


In [18]:
def DatasetToUserItemDataFrame(dataframe,userID,itemID,rating,countCol):
  #Setting new item IDs from string to int 
  itemKeys = [] 
  i = 0
  for item in dataframe.groupby(itemID).count()[countCol].values:
    temp = np.full((item),i)
    itemKeys = np.append(itemKeys,temp)
    i += 1

  #Setting new user IDs from string to int
  userKeysDic = {}
  userKeys = np.zeros((dataframe[userID].size))
  i = 0
  for user in dataframe[userID].unique():
    userKeysDic[user] = i
    i += 1
  i = 0
  for user in dataframe[userID]:
    userKeys[i] = userKeysDic[user]
    i += 1

  #Converting arrays from lfoat to int 
  userKeys = userKeys.astype(int)
  itemKeys = itemKeys.astype(int)

  
  user_item = csr_matrix((dataframe[rating].values.astype(int),(userKeys,itemKeys))) #Creating sparse matrix
  user_item_matrix = user_item.toarray() #Converting sparse matrix into array
  #df_user_item = pd.DataFrame(user_item_matrix,index = dataframe[userID].unique()  ,columns = dataframe[itemID].unique() ) 

  return user_item

In [19]:
user_item_matrix = DatasetToUserItemDataFrame(df_test,'User_ID','Item_ID','rating','User_ID')

In [20]:
user_item_matrix.shape

(23982, 30483)

In [21]:
#Train/Test data split
from sklearn.model_selection import train_test_split
X_train, X_test= train_test_split(user_item_matrix)

In [22]:
X_train = X_train.toarray()

In [23]:
X_train.shape

(17986, 30483)

In [48]:
X_train.shape[0] * X_train.shape[1]

548267238

In [ ]:
np.count_nonzero(X_train)/(X_train.shape[0]*X_train.shape[1]) * 100

In [24]:
X_test.shape

(5996, 30483)

In [25]:
#GMF + Autoencoder


encoder_input = layers.Input(shape=(X_train.shape[1]),name='user_item')
flat = layers.Flatten()(encoder_input)
hid_encoder = layers.Dense(512,activation="relu")(flat)
encoder_output = layers.Dense(256,activation="relu")(hid_encoder)

decoder_input = layers.Dense(512,activation="relu")(encoder_output)
decoder_output = layers.Dense(X_train.shape[1],activation="relu")(decoder_input)

autoencoder = tf.keras.Model(inputs = encoder_input, outputs = decoder_output)

In [28]:
autoencoder.compile(optimizer = 'adam',
                    loss = 'mae',
                    metrics=['accuracy'])

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.fit(X_train, X_train, epochs = 5)